In [1]:
import sys
import polite_pyalex
from pyalex import Works 
import pandas as pd

In [12]:
oa_long = ['W2088209891',
 'W2134049161',
 'W1767272795',
 'W2048185449',
 'W4232875366',
 'W2952984634',
 'W2115339903',
 'W2063332351',
 'W2160597895',
 'W2088909840',
 'W4291732787',
 'W1907242353',
 'W1976396629',
 'W3121567788',
 'W4254015553',
 'W2588027260',
 'W2951351472',
 'W2020123068',
 'W2006972602',
 'W2953072907',
 'W2566143661',
 'W2020807482',
 'W2345375849',
 'W2737712680',
 'W1997125517',
 'W2463568293',
 'W1970217003',
 'W2023783748',
 'W2322381034',
 'W2137725064',
 'W2015134878']
len(oa_long)

31

In [23]:
# from .polite_pyalex import Works

oa_id_list = ["W2741809807", "W1560783210", "W2029057325", "W2160597895"]
#print(oa_id_list)


def extract_refs(oa_id_list):
    """
    extracts references from manuscript (first layer) and references
    of those (second layer),
    based on OpenAlex ids retrieved from manuscript
    """

    ## first layer
    first_layer = []
    if len(oa_id_list) < 25: 
        first_layer = Works().filter(openalex="|".join(oa_id_list)).get()
        first_layer = [ref["referenced_works"] for ref in first_layer]
        first_layer = list(
            {i for lists in first_layer for i in lists}
            )  # list comprehension returning a set
    
    if len(oa_id_list) > 25:
        for ref in range(0, len(oa_id_list), 25):
            subset = '|'.join(str(id) for id in oa_id_list[ref : ref + 25])
            print("subset", subset)
            first_layer.append(Works().filter(openalex=subset).get())
            print(len(first_layer))
            print(first_layer)
        first_layer = [entry for lists in first_layer for entry in lists] 
        first_layer = [id["id"].split("/")[-1] for id in first_layer[0]]
        print(first_layer)
        print(len(first_layer))
        
    ## second layer
    second_layer = []
    for ref in range(0, len(first_layer), 25):
        subset = '|'.join(str(id) for id in first_layer[ref : ref + 25])
        second_layer.append(Works().filter(openalex=subset).get())

    return [entry for lists in second_layer for entry in lists]

refs = extract_refs(oa_long)


subset W2088209891|W2134049161|W1767272795|W2048185449|W4232875366|W2952984634|W2115339903|W2063332351|W2160597895|W2088909840|W4291732787|W1907242353|W1976396629|W3121567788|W4254015553|W2588027260|W2951351472|W2020123068|W2006972602|W2953072907|W2566143661|W2020807482|W2345375849|W2737712680|W1997125517
1
[[{'id': 'https://openalex.org/W2088209891', 'doi': 'https://doi.org/10.1126/science.149.3683.510', 'title': 'Networks of Scientific Papers', 'display_name': 'Networks of Scientific Papers', 'publication_year': 1965, 'publication_date': '1965-07-30', 'ids': {'openalex': 'https://openalex.org/W2088209891', 'doi': 'https://doi.org/10.1126/science.149.3683.510', 'mag': '2088209891', 'pmid': 'https://pubmed.ncbi.nlm.nih.gov/14325149'}, 'language': 'en', 'primary_location': {'is_oa': False, 'landing_page_url': 'https://doi.org/10.1126/science.149.3683.510', 'pdf_url': None, 'source': {'id': 'https://openalex.org/S3880285', 'display_name': 'Science', 'issn_l': '0036-8075', 'issn': ['0036-

TypeError: string indices must be integers

In [ ]:

def create_ref_csv(refs):
    """
    From OpenAlex references, extracts OpenAlex ID, abstracts,
    journal ISSN-L and all authors in correct order.
    Returns a dataframe
    """

    ids = [w["id"].split("/")[-1] for w in refs]
    abstracts = [w["abstract"] for w in refs]
    journal = [
        w["primary_location"]["source"]["issn_l"]
        for w in refs
        if w["primary_location"]["source"]
    ]
    authorships = [w["authorships"] for w in refs]
    authors = []
    for papers in authorships:
        current_paper = [author["author"]["display_name"] for author in papers]
        authors.append(current_paper)

    return pd.DataFrame(
        list(zip(ids, journal, authors, abstracts)),
        columns=["oa_id", "journal_issnl", "authors", "abstracts"],
    )


ref_csv = create_ref_csv(refs)
print(ref_csv)